In [ ]:
!pip3 install ../../random-forest-mc/

In [20]:
from random_forest_mc.model import RandomForestMC
from random_forest_mc.utils import LoadDicts
import pandas as pd
import numpy as np

from typing import Any
from typing import Dict
from typing import List
from typing import Optional
from typing import Tuple
from typing import Union

from numbers import Number

In [3]:
dicts = LoadDicts("../../random-forest-mc/tests/")
dataset_dict = dicts.datasets_metadata
ds_name = "titanic"
params = dataset_dict[ds_name]
dataset = (
    pd.read_csv(params["csv_path"])[params["ds_cols"] + [params["target_col"]]]
    .dropna()
    .reset_index(drop=True)
)
dataset["Age"] = dataset["Age"].astype(np.uint8)
dataset["SibSp"] = dataset["SibSp"].astype(np.uint8)
dataset["Pclass"] = dataset["Pclass"].astype(str)
dataset["Fare"] = dataset["Fare"].astype(np.uint32)
cls = RandomForestMC(target_col=params["target_col"])
cls.fit(dataset)

Planting the forest: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:06<00:00,  2.38it/s]


NameError: name 'check' is not defined

In [4]:
row = dataset.reset_index(drop=True).loc[0]
predict_row = cls.predict(row)
predict_ds = cls.predict(dataset.sample(n=10))
predict_probs_ds = cls.predict_proba(dataset.sample(n=10))

In [5]:
cls

RandomForestMC(len(Forest)=16,n_trees=16,model_version=1.0.4-dev,module_version=1.0.4-dev)

In [17]:
# a row of pd.DataFrame.iterrows()
# dsRow: TypeAlias = pd.core.series.Series
dsRow = pd.core.series.Series

# A tree composed by a assimetric tree of dictionaries:
# TypeTree: TypeAlias = Dict
TypeTree = Dict

# Value type of classes
# TypeClassVal: TypeAlias = Any
TypeClassVal = Any  # !Review if is not forced to be str!

# Type of the leaf
# TypeLeaf: TypeAlias = Dict[TypeClassVal, float]
TypeLeaf = Dict[TypeClassVal, float]

In [18]:
# How to format a dict with values to fill the missing ones
featName = str
featValue = Union[str, Number]
dictValues = Dict[featName: featValue]


class RandomForestMC_Ext(RandomForestMC):
    def _fill_row_missing(row: dsRow, dict_values : dictValues) -> pd.DataFrame:
        pass
        
    def predictMissingValues(row_or_matrix: Union[dsRow, pd.DataFrame], dict_values : dictValues):
        if isinstance(row_or_matrix, dsRow):
            pass
        if isinstance(row_or_matrix, pd.DataFrame):
            pass

In [9]:
cls = RandomForestMC_Ext(target_col=params["target_col"])

In [10]:
cls

RandomForestMC(len(Forest)=0,n_trees=16,model_version=1.0.4-dev,module_version=1.0.4-dev)

In [14]:
cls.__class__.__name__

'RandomForestMC_Ext'